# Complete BombCell + UnitMatch Pipeline Demo

This notebook demonstrates the complete pipeline for neural spike analysis using BombCell and UnitMatch:

## 🎯 **What This Demo Does**

### Part 1: BombCell Quality Control
- Analyzes spike sorting data from Kilosort
- Extracts comprehensive quality metrics for each unit
- Classifies units as "GOOD", "MUA", "NOISE", or "NON-SOMA"  
- **Creates waveform data specifically formatted for UnitMatch**

### Part 2: UnitMatch Cross-Session Tracking
- Uses BombCell outputs to track neurons across recording sessions
- Applies machine learning to match units based on waveform features
- Provides interactive tools for manual curation
- Generates probability matrices showing match confidence

## 🔗 **Key Integration Points**
- BombCell creates `RawWaveforms/` folders that UnitMatch requires
- Special parameters optimize BombCell for UnitMatch compatibility
- Quality classifications help filter units for reliable matching
- Both tools work seamlessly with standard Kilosort outputs

## 📋 **What You Need**
- Kilosort output directories from 2+ recording sessions if you want to track across days or for 1 recording session if you want to merge units intra-session
- Access to raw `.bin` files (for waveform extraction)
- Meta files (`.meta` for SpikeGLX or `.oebin` for Open Ephys)


In [2]:
# 📦 Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import BombCell for quality control
import bombcell as bc

# Import UnitMatch for cross-session tracking (optional - only if running Part 2)
try:
    import UnitMatchPy.bayes_functions as bf
    import UnitMatchPy.utils as util
    import UnitMatchPy.overlord as ov
    import UnitMatchPy.save_utils as su
    import UnitMatchPy.GUI as um_gui
    import UnitMatchPy.assign_unique_id as aid
    import UnitMatchPy.default_params as default_params
    UNITMATCH_AVAILABLE = True
    print("✅ BombCell and UnitMatch imported successfully")
except ImportError as e:
    UNITMATCH_AVAILABLE = False
    print("✅ BombCell imported successfully")
    print("⚠️  UnitMatch not available - please install: pip install UnitMatchPy")
    print(f"    Error: {e}")

print("🚀 Ready to analyze neural data!")

✅ BombCell and UnitMatch imported successfully
🚀 Ready to analyze neural data!


## 📁 Part 1: Configure Data Paths for BombCell

**Edit these paths to point to your actual data:**

For this demo, we'll analyze data from multiple sessions to demonstrate the complete pipeline.

In [3]:
# 📁 Configure Data Paths - Using Processing Playground Dataset

# These are the exact paths from the processing_playground for testing
# calca_302 sessions from 2023-04-19 and 2023-04-20

session_configs = [
    {
        'name': 'calca_302_2023-04-19',
        'ks_dir': r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/kilosort4',
        'raw_file': r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/cz_npxl_g0_t0.imec0.ap.bin',
        'meta_file': r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/cz_npxl_g0_t0.imec0.ap.meta'
    },
    {
        'name': 'calca_302_2023-04-20', 
        'ks_dir': r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-20/cz_npxl_g0/cz_npxl_g0_imec0/kilosort4',
        'raw_file': r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-20/cz_npxl_g0/cz_npxl_g0_imec0/cz_npxl_g0_t0.imec0.ap.bin',
        'meta_file': r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-20/cz_npxl_g0/cz_npxl_g0_imec0/cz_npxl_g0_t0.imec0.ap.meta'
    }
]

kilosort_version = 4

print("🎯 BombCell + UnitMatch Pipeline Demo")
print("📊 Dataset: calca_302 cross-session tracking")
print(f"🔬 Sessions to analyze: {len(session_configs)}")
for i, config in enumerate(session_configs):
    print(f"   Session {i+1}: {config['name']}")
    print(f"      📁 KS: {Path(config['ks_dir']).name}")
    
print(f"\n🔧 Kilosort version: {kilosort_version}")
print("🎯 This demo will run BombCell first, then use outputs for UnitMatch tracking")

🎯 BombCell + UnitMatch Pipeline Demo
📊 Dataset: calca_302 cross-session tracking
🔬 Sessions to analyze: 2
   Session 1: calca_302_2023-04-19
      📁 KS: kilosort4
   Session 2: calca_302_2023-04-20
      📁 KS: kilosort4

🔧 Kilosort version: 4
🎯 This demo will run BombCell first, then use outputs for UnitMatch tracking


## 🧪 Part 1: Run BombCell Quality Control

**What BombCell does:**
- Analyzes each unit's waveform properties, firing patterns, and spatial characteristics
- Calculates 15+ quality metrics (amplitude, drift, contamination, etc.)
- Classifies units into categories based on quality thresholds
- **Special for UnitMatch**: Extracts 1000 raw spikes per unit (vs. standard 100)

**Key UnitMatch optimizations:**
- `detrendWaveform=False`: Preserves raw waveform shape for matching
- `nRawSpikesToExtract=1000`: More spikes = better cross-validation
- `saveMultipleRaw=True`: Saves separate waveform sets for validation

In [ ]:
def run_bombcell_session(session_config):
    """Run BombCell on a single session with UnitMatch parameters"""
    
    name = session_config['name']
    ks_dir = session_config['ks_dir']
    raw_file = session_config.get('raw_file')
    meta_file = session_config.get('meta_file')
    
    # Create save path in the kilosort directory
    save_path = Path(ks_dir) / "bombcell_testing_jf"
    
    print(f"🔬 Analyzing session: {name}")
    print(f"   📁 Kilosort directory: {ks_dir}")
    print(f"   🗂️  Raw file: {Path(raw_file).name if raw_file else 'Not specified'}")
    print(f"   📄 Meta file: {Path(meta_file).name if meta_file else 'Not specified'}")
    print(f"   💾 Results will be saved to: {save_path}")
    
    # Check if BombCell has already been run
    existing_results = save_path / "cluster_bc_unitType.tsv"
    if existing_results.exists():
        print(f"   ✅ Found existing BombCell results - loading from disk")
        
        # Load existing results
        try:
            param, quality_metrics, unit_type_string = bc.load_bc_results(str(save_path))
            unit_type = np.array([1 if ut == 'GOOD' else 0 for ut in unit_type_string])
            
            print(f"   📊 Loaded results - Total units: {len(unit_type)}")
            print(f"   🟢 Good units: {np.sum(unit_type_string == 'GOOD')}")
            print(f"   🟡 MUA units: {np.sum(unit_type_string == 'MUA')}")
            print(f"   🔴 Noise units: {np.sum(unit_type_string == 'NOISE')}")
            print(f"   ⚫ Non-somatic: {np.sum(unit_type_string == 'NON-SOMA')}")
            
        except Exception as e:
            print(f"   ⚠️  Error loading existing results: {e}")
            print(f"   🔄 Will re-run BombCell analysis...")
            existing_results = None
    else:
        existing_results = None
    
    # Run BombCell if no existing results or loading failed
    if not existing_results or not existing_results.exists():
        print("   🚀 Running BombCell analysis...")
        quality_metrics, param, unit_type, unit_type_string = bc.run_bombcell_unit_match(
            ks_dir=ks_dir,
            save_path=str(save_path),
            raw_file=raw_file,
            meta_file=meta_file,
            kilosort_version=kilosort_version
        )
        
        print(f"   ✅ Analysis complete!")
        print(f"   📊 Total units: {len(unit_type)}")
        print(f"   🟢 Good units: {np.sum(unit_type_string == 'GOOD')}")
        print(f"   🟡 MUA units: {np.sum(unit_type_string == 'MUA')}")
        print(f"   🔴 Noise units: {np.sum(unit_type_string == 'NOISE')}")
        print(f"   ⚫ Non-somatic: {np.sum(unit_type_string == 'NON-SOMA')}")
    
    # Check UnitMatch waveforms
    raw_waveforms_dir = save_path / "RawWaveforms"
    if raw_waveforms_dir.exists():
        waveform_files = list(raw_waveforms_dir.glob("Unit*_RawSpikes.npy"))
        print(f"   🎯 UnitMatch waveforms: {len(waveform_files)} files saved")
    else:
        print(f"   ⚠️  No UnitMatch waveforms - check raw file access")
    
    return {
        'name': name,
        'ks_dir': ks_dir,
        'save_path': str(save_path),
        'quality_metrics': quality_metrics,
        'param': param,
        'unit_type': unit_type,
        'unit_type_string': unit_type_string
    }

# Run BombCell on all sessions
print("🚀 Running BombCell analysis on all sessions...")
session_results = []

for i, session_config in enumerate(session_configs):
    print(f"\n{'='*60}")
    print(f"SESSION {i+1}/{len(session_configs)}")
    print(f"{'='*60}")
    result = run_bombcell_session(session_config)
    session_results.append(result)

print(f"\n🎉 BombCell analysis complete for {len(session_results)} sessions!")

🚀 Running BombCell analysis on all sessions...

SESSION 1/2
🔬 Analyzing session: calca_302_2023-04-19
   📁 Kilosort directory: /home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/kilosort4
   🗂️  Raw file: cz_npxl_g0_t0.imec0.ap.bin
   📄 Meta file: cz_npxl_g0_t0.imec0.ap.meta
   💾 Results will be saved to: /home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/kilosort4/bombcell_testing_jf
   🚀 Running BombCell analysis...
Using raw data cz_npxl_g0_t0.imec0.ap.bin.
🚀 Running BombCell with UnitMatch parameters...
   - Extracting 1000 raw spikes per unit
   - Saving multiple raw waveforms: True
   - Detrending waveforms: True
🚀 Starting BombCell quality metrics pipeline...
📁 Processing data from: /home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/kilosort4
Results will be saved to: /home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/kil

0it [00:00, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


## 📊 Part 1 Results: BombCell Quality Metrics Summary

Let's examine the quality control results and understand what BombCell found.

In [ ]:
# 📈 Analyze BombCell Results
for i, result in enumerate(session_results):
    print(f"\n{'='*60}")
    print(f"📊 SESSION {i+1}: {Path(result['ks_dir']).name}")
    print(f"{'='*60}")
    
    # Convert to DataFrame for easier analysis
    df = pd.DataFrame(result['quality_metrics'])
    df['unit_type'] = result['unit_type_string']
    df['cluster_id'] = df['phy_clusterID'].astype(int)
    
    # Unit type summary
    print("\n🏷️  Unit Classifications:")
    type_counts = df['unit_type'].value_counts()
    for unit_type, count in type_counts.items():
        percentage = (count / len(df)) * 100
        print(f"   {unit_type}: {count} units ({percentage:.1f}%)")
    
    # Key quality metrics summary
    print(f"\n🔍 Quality Metrics Summary:")
    good_units = df[df['unit_type'] == 'GOOD']
    if len(good_units) > 0:
        print(f"   📶 Amplitude (good units): {good_units['amplitude'].mean():.1f} ± {good_units['amplitude'].std():.1f} μV")
        print(f"   ⚡ Firing rate (good units): {good_units['fr_Hz'].mean():.2f} ± {good_units['fr_Hz'].std():.2f} Hz")
        print(f"   🎯 Presence ratio (good units): {good_units['presenceRatio'].mean():.3f} ± {good_units['presenceRatio'].std():.3f}")
        print(f"   🚫 RPV contamination (good units): {good_units['fractionRPVs_estimatedTauR'].mean()*100:.2f}% ± {good_units['fractionRPVs_estimatedTauR'].std()*100:.2f}%")
    
    # Check UnitMatch readiness
    raw_waveforms_dir = Path(result['save_path']) / "RawWaveforms"
    if raw_waveforms_dir.exists():
        waveform_files = list(raw_waveforms_dir.glob("Unit*_RawSpikes.npy"))
        print(f"\n🎯 UnitMatch Readiness:")
        print(f"   ✅ Raw waveforms extracted: {len(waveform_files)} units")
        print(f"   📁 Location: {raw_waveforms_dir}")
        
        # Check file sizes to verify extraction quality
        if waveform_files:
            sample_file = np.load(waveform_files[0])
            print(f"   📏 Waveform dimensions: {sample_file.shape} (channels × time × spikes)")
            print(f"   💾 Expected ~1000 spikes per unit for UnitMatch")
    else:
        print(f"\n⚠️  UnitMatch waveforms not found - may need raw data access")
    
    # Display sample of results
    print(f"\n📋 Sample Quality Metrics (first 5 units):")
    display_cols = ['cluster_id', 'unit_type', 'amplitude', 'fr_Hz', 'presenceRatio', 'fractionRPVs_estimatedTauR']
    display(df[display_cols].head())

In [ ]:
# 🎨 Create BombCell Summary Plots
print("🎨 Generating BombCell summary plots...")

for i, result in enumerate(session_results):
    print(f"\n📊 Plotting results for Session {i+1}")
    
    # Load the saved results for plotting
    quality_metrics = result['quality_metrics']
    param = result['param']
    
    # Create summary plots using BombCell's built-in plotting
    try:
        fig = bc.plot_summary_data(param, quality_metrics, 
                                   save_figure=True, 
                                   save_path=result['save_path'])
        plt.show()
        print(f"   ✅ Summary plots saved to: {result['save_path']}")
    except Exception as e:
        print(f"   ⚠️  Error creating plots: {e}")
        
        # Create basic plots manually if automated plotting fails
        df = pd.DataFrame(quality_metrics)
        df['unit_type'] = result['unit_type_string']
        
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        fig.suptitle(f"Session {i+1}: BombCell Quality Metrics", fontsize=14)
        
        # Amplitude distribution by unit type
        for unit_type in df['unit_type'].unique():
            subset = df[df['unit_type'] == unit_type]
            axes[0,0].hist(subset['amplitude'], alpha=0.6, label=unit_type, bins=20)
        axes[0,0].set_xlabel('Amplitude (μV)')
        axes[0,0].set_ylabel('Count')
        axes[0,0].set_title('Amplitude Distribution')
        axes[0,0].legend()
        
        # Firing rate vs presence ratio
        colors = {'GOOD': 'green', 'MUA': 'orange', 'NOISE': 'red', 'NON-SOMA': 'black'}
        for unit_type in df['unit_type'].unique():
            subset = df[df['unit_type'] == unit_type]
            axes[0,1].scatter(subset['presenceRatio'], subset['fr_Hz'], 
                            alpha=0.6, label=unit_type, c=colors.get(unit_type, 'gray'))
        axes[0,1].set_xlabel('Presence Ratio')
        axes[0,1].set_ylabel('Firing Rate (Hz)')
        axes[0,1].set_title('Firing Rate vs Presence Ratio')
        axes[0,1].legend()
        
        # RPV contamination
        rpv_data = df['fractionRPVs_estimatedTauR'] * 100
        rpv_data = rpv_data[~np.isnan(rpv_data)]
        axes[1,0].hist(rpv_data, bins=20, alpha=0.7)
        axes[1,0].set_xlabel('RPV Contamination (%)')
        axes[1,0].set_ylabel('Count')
        axes[1,0].set_title('Refractory Period Violations')
        
        # Unit type pie chart
        type_counts = df['unit_type'].value_counts()
        axes[1,1].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
        axes[1,1].set_title('Unit Type Distribution')
        
        plt.tight_layout()
        plt.show()

print("🎨 Plotting complete!")

## 🎯 Part 2: UnitMatch Cross-Session Tracking

**What UnitMatch does:**
- Compares units across different recording sessions using multiple features
- Uses machine learning (Naive Bayes) to calculate match probabilities  
- Identifies the same neurons recorded on different days
- Provides tools for manual curation and validation

**This section requires multiple sessions with BombCell outputs!**

In [ ]:
# 🔗 Setup UnitMatch Parameters and Paths

print("🎯 Setting up UnitMatch for cross-session tracking...")

# Get default UnitMatch parameters
um_param = default_params.get_default_param()

# Set up paths from our BombCell results
KS_dirs = [result['ks_dir'] for result in session_results]
um_param['KS_dirs'] = KS_dirs

# Set up BombCell paths - using exact paths from processing_playground
custom_bombcell_paths = [
    r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/bombcell_testing_jf/cluster_bc_unitType.tsv',
    r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-20/cz_npxl_g0/cz_npxl_g0_imec0/bombcell_testing_jf/cluster_bc_unitType.tsv'
]

custom_raw_waveform_paths = [
    r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-19/cz_npxl_g0/cz_npxl_g0_imec0/bombcell_testing_jf/RawWaveforms',
    r'/home/jf5479/cup/Chris/data/cta_backwards/calca_302/2023-04-20/cz_npxl_g0/cz_npxl_g0_imec0/bombcell_testing_jf/RawWaveforms'
]

print(f"📁 Kilosort directories: {len(KS_dirs)}")
for i, ks_dir in enumerate(KS_dirs):
    print(f"   Session {i+1}: {Path(ks_dir).name}")

print(f"📊 BombCell unit classifications:")
for i, bc_path in enumerate(custom_bombcell_paths):
    exists = "✅" if Path(bc_path).exists() else "❌"
    print(f"   Session {i+1}: {exists} {Path(bc_path).name}")
    
print(f"🎯 Raw waveforms for UnitMatch:")
for i, wv_path in enumerate(custom_raw_waveform_paths):
    exists = "✅" if Path(wv_path).exists() else "❌"
    n_files = len(list(Path(wv_path).glob("Unit*_RawSpikes.npy"))) if Path(wv_path).exists() else 0
    print(f"   Session {i+1}: {exists} {n_files} waveform files")

# Setup UnitMatch paths - this matches exactly what processing_playground does
try:
    wave_paths, unit_label_paths, channel_pos = util.paths_from_KS(
        KS_dirs, 
        custom_raw_waveform_paths=custom_raw_waveform_paths,
        custom_bombcell_paths=custom_bombcell_paths
    )
    
    um_param = util.get_probe_geometry(channel_pos[0], um_param)
    print("✅ UnitMatch paths configured successfully")
    UNITMATCH_READY = True
    
except Exception as e:
    print(f"❌ Error setting up UnitMatch paths: {e}")
    print("   Make sure BombCell has been run and waveforms extracted")
    UNITMATCH_READY = False

In [ ]:
# 🧠 Run UnitMatch Analysis Pipeline

if UNITMATCH_READY:
    print("🚀 Running UnitMatch cross-session analysis...")
    
    # Step 0: Load good units and waveform data
    print("📊 Step 0: Loading unit waveforms...")
    waveform, session_id, session_switch, within_session, good_units, um_param = util.load_good_waveforms(
        wave_paths, unit_label_paths, um_param, good_units_only=True
    ) 
    
    print(f"   ✅ Loaded {len(np.concatenate(good_units))} good units across {len(session_results)} sessions")
    print(f"   📏 Waveform shape: {waveform.shape} (units × channels × time)")
    
    # Create cluster info
    clus_info = {
        'good_units': good_units, 
        'session_switch': session_switch, 
        'session_id': session_id,
        'original_ids': np.concatenate(good_units)
    }
    
    # Step 1: Extract waveform parameters
    print("🔍 Step 1: Extracting waveform features...")
    extracted_wave_properties = ov.extract_parameters(waveform, channel_pos, clus_info, um_param)
    print("   ✅ Extracted amplitude, spatial decay, and waveform features")
    
    # Steps 2-4: Calculate similarity metrics with drift correction
    print("📐 Steps 2-4: Computing similarity metrics and drift correction...")
    total_score, candidate_pairs, scores_to_include, predictors = ov.extract_metric_scores(
        extracted_wave_properties, session_switch, within_session, um_param, niter=2
    )
    print(f"   ✅ Found {np.sum(candidate_pairs)} candidate unit pairs")
    print(f"   📊 Metrics included: {list(scores_to_include.keys())}")
    
    # Step 5: Probability analysis using Naive Bayes
    print("🧮 Step 5: Calculating match probabilities...")
    
    # Set up priors
    prior_match = 1 - (um_param['n_expected_matches'] / um_param['n_units']**2)
    priors = np.array([prior_match, 1 - prior_match])
    
    # Train Naive Bayes classifier
    labels = candidate_pairs.astype(int)
    cond = np.unique(labels)
    parameter_kernels = bf.get_parameter_kernels(scores_to_include, labels, cond, um_param, add_one=1)
    
    # Calculate match probabilities
    probability = bf.apply_naive_bayes(parameter_kernels, priors, predictors, um_param, cond)
    output_prob_matrix = probability[:, 1].reshape(um_param['n_units'], um_param['n_units'])
    
    print(f"   ✅ Calculated probability matrix: {output_prob_matrix.shape}")
    print(f"   📈 Max probability: {np.max(output_prob_matrix):.3f}")
    print(f"   📊 Mean probability: {np.mean(output_prob_matrix):.3f}")
    
else:
    print("⏭️  Skipping UnitMatch analysis - requirements not met")

In [ ]:
# 📊 Analyze UnitMatch Results

if UNITMATCH_READY:
    print("📈 Analyzing UnitMatch results...")
    
    # Evaluate matching performance
    match_threshold = um_param.get('match_threshold', 0.75)
    util.evaluate_output(output_prob_matrix, um_param, within_session, session_switch, 
                        match_threshold=match_threshold)
    
    # Create binary match matrix
    output_threshold = np.zeros_like(output_prob_matrix)
    output_threshold[output_prob_matrix > match_threshold] = 1
    
    # Count matches
    total_matches = np.sum(output_threshold)
    within_session_matches = np.sum(output_threshold * within_session)
    cross_session_matches = total_matches - within_session_matches
    
    print(f"\n🎯 Match Summary (threshold = {match_threshold}):")
    print(f"   🔗 Total matches found: {total_matches}")
    print(f"   📍 Within-session matches: {within_session_matches}")
    print(f"   🌉 Cross-session matches: {cross_session_matches}")
    
    # Visualize probability matrix
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Probability matrix
    im1 = axes[0].imshow(output_prob_matrix, cmap='viridis', aspect='auto')
    axes[0].set_title('Unit Match Probability Matrix')
    axes[0].set_xlabel('Unit Index')
    axes[0].set_ylabel('Unit Index')
    plt.colorbar(im1, ax=axes[0], label='Match Probability')
    
    # Session boundaries
    n_units_cumsum = np.cumsum([0] + [len(units) for units in good_units])
    for boundary in n_units_cumsum[1:-1]:
        axes[0].axhline(boundary, color='red', linestyle='--', alpha=0.7)
        axes[0].axvline(boundary, color='red', linestyle='--', alpha=0.7)
    
    # Binary matches
    im2 = axes[1].imshow(output_threshold, cmap='Greys', aspect='auto')
    axes[1].set_title(f'Matches Above Threshold ({match_threshold})')
    axes[1].set_xlabel('Unit Index')
    axes[1].set_ylabel('Unit Index')
    
    # Session boundaries
    for boundary in n_units_cumsum[1:-1]:
        axes[1].axhline(boundary, color='red', linestyle='--', alpha=0.7)
        axes[1].axvline(boundary, color='red', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Red dashed lines show session boundaries")
    print("🎯 Diagonal represents within-session matches")
    print("🌉 Off-diagonal represents cross-session matches")
    
else:
    print("⏭️  No UnitMatch results to analyze")

In [ ]:
# 🎮 Interactive Manual Curation with UnitMatch GUI

if UNITMATCH_READY:
    print("🎮 Setting up interactive curation tools...")
    
    # Prepare data for GUI - extract all the required variables
    amplitude = extracted_wave_properties['amplitude']
    spatial_decay = extracted_wave_properties['spatial_decay']
    avg_centroid = extracted_wave_properties['avg_centroid']
    avg_waveform = extracted_wave_properties['avg_waveform']
    avg_waveform_per_tp = extracted_wave_properties['avg_waveform_per_tp']
    wave_idx = extracted_wave_properties['good_wave_idxs']
    max_site = extracted_wave_properties['max_site']
    max_site_mean = extracted_wave_properties['max_site_mean']
    
    # Process info for GUI
    um_gui.process_info_for_GUI(
        output_prob_matrix, match_threshold, scores_to_include, total_score, 
        amplitude, spatial_decay, avg_centroid, avg_waveform, avg_waveform_per_tp, 
        wave_idx, max_site, max_site_mean, waveform, within_session, 
        channel_pos, clus_info, um_param
    )
    
    print("✅ GUI data prepared successfully")
    print("\n🎯 To launch interactive curation:")
    print("   Run: is_match, not_match, matches_GUI = um_gui.run_GUI()")
    print("\n🔍 The GUI provides:")
    print("   - Interactive visualization of unit pairs")
    print("   - Side-by-side waveform comparisons")
    print("   - Quality metric displays")
    print("   - Manual accept/reject controls")
    print("   - Real-time match probability updates")
    
    print("\n💡 Curation workflow:")
    print("   1. GUI shows potential matches above threshold")
    print("   2. Review waveform similarity and spatial locations")
    print("   3. Accept good matches, reject false positives")
    print("   4. Use curated results for final unit tracking")
    
else:
    print("⏭️  GUI setup skipped - UnitMatch analysis not ready")

In [ ]:
# 💾 Save Results and Generate Final Outputs

if UNITMATCH_READY:
    print("💾 Saving UnitMatch results and generating final outputs...")
    
    # Assign unique IDs to matched units across sessions
    UIDs = aid.assign_unique_id(output_prob_matrix, um_param, clus_info)
    
    # Get final matches above threshold
    matches = np.argwhere(output_threshold == 1)
    
    # Create save directory next to the first session's BombCell results
    save_dir = Path(session_results[0]['save_path']).parent / "UnitMatch_Results"
    save_dir.mkdir(exist_ok=True)
    
    print(f"📁 Saving results to: {save_dir}")
    
    # Save comprehensive UnitMatch results
    su.save_to_output(
        str(save_dir), 
        scores_to_include, 
        matches,
        output_prob_matrix, 
        avg_centroid, 
        avg_waveform, 
        avg_waveform_per_tp, 
        max_site,
        total_score, 
        output_threshold, 
        clus_info, 
        um_param, 
        UIDs=UIDs, 
        matches_curated=None,  # Set to matches_curated if manual curation was performed
        save_match_table=True
    )
    
    print("✅ Results saved successfully!")
    
    # Generate summary statistics
    n_unique_neurons = len(np.unique(UIDs))
    n_total_units = len(np.concatenate(good_units))
    n_cross_session_matches = cross_session_matches
    tracking_efficiency = (n_cross_session_matches / min([len(units) for units in good_units])) * 100
    
    print(f"\n📊 Final Results Summary:")
    print(f"{'='*60}")
    print(f"🧠 Dataset: calca_302 (2 sessions)")
    print(f"📊 Total units analyzed: {sum(len(result['unit_type']) for result in session_results)}")
    print(f"✅ Good units tracked: {n_total_units}")
    print(f"🔗 Cross-session matches found: {n_cross_session_matches}")
    print(f"🏷️  Unique neurons identified: {n_unique_neurons}")
    print(f"📈 Tracking efficiency: {tracking_efficiency:.1f}%")
    
    print(f"\n📁 Output Files Generated:")
    print(f"   📊 output_prob_matrix.npy - Match probability matrix")
    print(f"   📋 match_table.csv - Detailed match information")
    print(f"   🏷️  unique_IDs.npy - Cross-session unit identifiers")
    print(f"   📈 total_score.npy - Combined similarity scores")
    print(f"   🎯 clus_info.json - Unit metadata and session info")
    
    print(f"\n🎯 Next Steps:")
    print(f"   1. Review match probabilities and validate results")
    print(f"   2. Use unique IDs for longitudinal analysis")
    print(f"   3. Analyze stability of tracked neurons")
    print(f"   4. Examine changes in firing patterns across sessions")
    
else:
    print("⏭️  No UnitMatch results to save - analysis was not completed")

## 🎉 Pipeline Complete!


### Key Files Generated

**BombCell Outputs (per session):**
- `qMetrics/cluster_bc_unitType.tsv` - Unit classifications
- `qMetrics/templates._bc_qMetrics.parquet` - Quality metrics
- `qMetrics/RawWaveforms/` - Raw spike waveforms for UnitMatch
- `qMetrics/summary_plots.png` - Quality control visualizations

**UnitMatch Outputs:**
- `UnitMatch_Results/output_prob_matrix.npy` - Match probabilities
- `UnitMatch_Results/match_table.csv` - Unit matches and scores
- `UnitMatch_Results/unique_IDs.npy` - Cross-session unit identifiers
